In [ ]:
import os, sys, re, json, subprocess
from datetime import datetime, date

# ----------------- RUTAS (ajusta si cambias tu carpeta) -----------------
BASE_DIR = r"C:\Users\Jorge Vasquez\DocoBr"
TEMPLATE_DOCX = os.path.join(BASE_DIR, "main.docx")
INDEX_HTML = os.path.join(BASE_DIR, "index.html")
OUT_DIR = os.path.join(BASE_DIR, "GENERADOS")

os.makedirs(OUT_DIR, exist_ok=True)

# ----------------- DEPENDENCIAS -----------------
def _pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "--quiet"] + pkgs
    subprocess.check_call(cmd)

try:
    from flask import Flask, request, send_file, jsonify
except ImportError:
    _pip_install(["flask"])
    from flask import Flask, request, send_file, jsonify

try:
    from docxtpl import DocxTemplate
except ImportError:
    _pip_install(["docxtpl"])
    from docxtpl import DocxTemplate

# ----------------- UTILIDADES -----------------
MESES_ES = {
    1:"enero",2:"febrero",3:"marzo",4:"abril",5:"mayo",6:"junio",
    7:"julio",8:"agosto",9:"septiembre",10:"octubre",11:"noviembre",12:"diciembre"
}

def _fmt_fecha_es(fecha_str: str) -> str:
    """
    Acepta:
      - ISO 'YYYY-MM-DD' (desde input type=date)
      - 'DD/MM/YYYY'
      - o cualquier texto ya listo (si no reconoce, lo devuelve tal cual)
    Devuelve: '21 de enero de 2026'
    """
    if not fecha_str:
        return ""
    s = str(fecha_str).strip()

    # ISO
    m = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", s)
    if m:
        y, mo, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        return f"{d} de {MESES_ES.get(mo, mo)} de {y}"

    # DD/MM/YYYY
    m = re.fullmatch(r"(\d{2})/(\d{2})/(\d{4})", s)
    if m:
        d, mo, y = int(m.group(1)), int(m.group(2)), int(m.group(3))
        return f"{d} de {MESES_ES.get(mo, mo)} de {y}"

    # Ya viene formateado
    return s

def _safe_filename(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"[^\w\-]+", "_", s, flags=re.UNICODE)
    return s or "SIN_NOMBRE"

# ----------------- GENERAR index.html -----------------
html = r"""<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>DocoBr - Constancia de No Adeudo</title>
  <style>
    body { font-family: Arial, sans-serif; padding: 24px; max-width: 980px; margin: auto; background:#fafafa;}
    h1 { margin: 0 0 6px 0; }
    p.sub { margin: 0 0 18px 0; color:#555; }
    .card { background:#fff; border:1px solid #e6e6e6; border-radius:14px; padding:16px; box-shadow: 0 2px 10px rgba(0,0,0,.04); }
    .grid { display:grid; grid-template-columns: 1fr 1fr; gap:12px; }
    label { font-weight:700; display:block; margin-bottom:6px; }
    input, textarea, select {
      width:100%; padding:10px; border:1px solid #cfcfcf; border-radius:10px; font-size:14px;
      outline:none;
    }
    input:focus, textarea:focus { border-color:#7aa7ff; box-shadow:0 0 0 3px rgba(122,167,255,.18); }
    .actions { display:flex; gap:10px; margin-top:14px; align-items:center; flex-wrap:wrap;}
    button {
      padding:12px 16px; border:0; border-radius:12px; cursor:pointer; font-weight:800;
    }
    button.primary { background:#111827; color:#fff; }
    button.secondary { background:#e5e7eb; color:#111827; }
    .hint { font-size: 12px; color:#666; margin-top:6px; }
    #msg { margin-top:12px; font-weight:700; }
    .err { color:#b91c1c; }
    .ok { color:#047857; }
    .small { font-size:12px; color:#666; margin-top:8px;}
    @media (max-width: 820px){
      .grid { grid-template-columns: 1fr; }
    }
  </style>
</head>
<body>
  <h1>Generador de Constancia de No Adeudo</h1>
  <p class="sub">Rellena el formulario y genera el Word usando <code>main.docx</code>.</p>

  <div class="card">
    <div class="grid">
      <div>
        <label>Nombre completo ({{NOMBRE}})</label>
        <input id="NOMBRE" placeholder="Ej: Juan Carlos Pérez López" />
      </div>

      <div>
        <label>DNI ({{DNI}})</label>
        <input id="DNI" inputmode="numeric" placeholder="Ej: 45879632" maxlength="8" />
        <div class="hint">8 dígitos</div>
      </div>

      <div>
        <label>Entidad ({{ENTIDAD}})</label>
        <input id="ENTIDAD" placeholder="Ej: Banco XYZ S.A." />
      </div>

      <div>
        <label>Crédito(s) ({{CREDITOS}})</label>
        <input id="CREDITOS" placeholder="Ej: 123456 / 789012" />
      </div>

      <div>
        <label>Fecha de castigo / transferencia ({{FECHA_CASTIGO}})</label>
        <input id="FECHA_CASTIGO" type="date" />
        <div class="hint">Se convertirá a formato: “21 de enero de 2026”.</div>
      </div>

      <div>
        <label>Fecha de pago ({{FECHA_PAGO}})</label>
        <input id="FECHA_PAGO" type="date" />
      </div>

      <div style="grid-column:1/-1;">
        <label>Fecha de emisión ({{FECHA_HOY}})</label>
        <input id="FECHA_HOY" type="date" />
        <div class="hint">Opcional: si lo dejas vacío, se usará la fecha de hoy.</div>
      </div>
    </div>

    <div class="actions">
      <button class="primary" id="btnProcesar">PROCESAR</button>
      <button class="secondary" id="btnLimpiar" type="button">LIMPIAR</button>
    </div>

    <div id="msg"></div>
    <div class="small">El archivo generado también se guardará en <b>GENERADOS</b> (en tu carpeta DocoBr).</div>
  </div>

  <script>
    const $ = (id) => document.getElementById(id);

    function setMsg(text, ok=true){
      const el = $("msg");
      el.textContent = text;
      el.className = ok ? "ok" : "err";
    }

    function validar(data){
      if(!data.NOMBRE.trim()) return "Falta NOMBRE";
      if(!/^\d{8}$/.test(data.DNI.trim())) return "DNI debe tener 8 dígitos";
      if(!data.ENTIDAD.trim()) return "Falta ENTIDAD";
      if(!data.CREDITOS.trim()) return "Falta CREDITOS";
      if(!data.FECHA_CASTIGO.trim()) return "Falta FECHA_CASTIGO";
      if(!data.FECHA_PAGO.trim()) return "Falta FECHA_PAGO";
      return null;
    }

    $("btnLimpiar").addEventListener("click", () => {
      ["NOMBRE","DNI","ENTIDAD","CREDITOS","FECHA_CASTIGO","FECHA_PAGO","FECHA_HOY"].forEach(k => $(k).value = "");
      setMsg("");
    });

    $("btnProcesar").addEventListener("click", async () => {
      const data = {
        NOMBRE: $("NOMBRE").value || "",
        DNI: $("DNI").value || "",
        ENTIDAD: $("ENTIDAD").value || "",
        CREDITOS: $("CREDITOS").value || "",
        FECHA_CASTIGO: $("FECHA_CASTIGO").value || "",
        FECHA_PAGO: $("FECHA_PAGO").value || "",
        FECHA_HOY: $("FECHA_HOY").value || ""
      };

      const err = validar(data);
      if(err){ setMsg(err, false); return; }

      setMsg("Generando documento...");

      try{
        const resp = await fetch("/procesar", {
          method: "POST",
          headers: {"Content-Type":"application/json"},
          body: JSON.stringify(data)
        });

        if(!resp.ok){
          const e = await resp.json().catch(()=>({}));
          throw new Error(e.message || "Error desconocido");
        }

        const blob = await resp.blob();
        const url = URL.createObjectURL(blob);

        const a = document.createElement("a");
        a.href = url;
        a.download = `Constancia_No_Adeudo_${data.DNI}.docx`;
        document.body.appendChild(a);
        a.click();
        a.remove();

        URL.revokeObjectURL(url);
        setMsg("Listo. Se descargó el Word y se guardó copia en GENERADOS.");
      }catch(e){
        setMsg("Error: " + e.message, false);
      }
    });
  </script>
</body>
</html>
"""

with open(INDEX_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ index.html generado/actualizado en: {INDEX_HTML}")
print(f"📄 Plantilla esperada: {TEMPLATE_DOCX}")
print(f"📁 Carpeta de salida: {OUT_DIR}")

# ----------------- SERVIDOR WEB (Flask) -----------------
app = Flask(__name__, static_folder=BASE_DIR, static_url_path="")

@app.get("/")
def home():
    # devuelve index.html desde la carpeta BASE_DIR
    return app.send_static_file("index.html")

@app.post("/procesar")
def procesar():
    if not os.path.exists(TEMPLATE_DOCX):
        return jsonify({"ok": False, "message": "No se encuentra main.docx en la carpeta DocoBr."}), 400

    data = request.get_json(force=True, silent=True) or {}

    # Normaliza/valida mínimos
    nombre = (data.get("NOMBRE") or "").strip()
    dni = (data.get("DNI") or "").strip()
    entidad = (data.get("ENTIDAD") or "").strip()
    creditos = (data.get("CREDITOS") or "").strip()

    if not nombre or not entidad or not creditos or not re.fullmatch(r"\d{8}", dni):
        return jsonify({"ok": False, "message": "Datos inválidos: revisa NOMBRE/DNI/ENTIDAD/CREDITOS."}), 400

    # Fechas: si FECHA_HOY viene vacío => hoy
    fecha_hoy_raw = (data.get("FECHA_HOY") or "").strip()
    if not fecha_hoy_raw:
        today = date.today()
        fecha_hoy_raw = f"{today.year:04d}-{today.month:02d}-{today.day:02d}"

    context = {
        "NOMBRE": nombre,
        "DNI": dni,
        "ENTIDAD": entidad,
        "CREDITOS": creditos,
        "FECHA_CASTIGO": _fmt_fecha_es((data.get("FECHA_CASTIGO") or "").strip()),
        "FECHA_PAGO": _fmt_fecha_es((data.get("FECHA_PAGO") or "").strip()),
        "FECHA_HOY": _fmt_fecha_es(fecha_hoy_raw),
    }

    # Render DOCX desde plantilla
    try:
        doc = DocxTemplate(TEMPLATE_DOCX)
        doc.render(context)

        out_name = f"Constancia_No_Adeudo_{_safe_filename(dni)}.docx"
        out_path = os.path.join(OUT_DIR, out_name)
        doc.save(out_path)

        # Devuelve el archivo (para descargar) y deja copia en GENERADOS
        return send_file(
            out_path,
            as_attachment=True,
            download_name=out_name,
            mimetype="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
        )
    except Exception as e:
        return jsonify({
            "ok": False,
            "message": "Error generando el documento. Usualmente es por placeholders 'partidos' en Word o etiquetas no encontradas.",
            "details": str(e)
        }), 400

# Ejecuta servidor
print("\n🚀 Web lista. Abre en tu navegador:")
print("   http://127.0.0.1:5000")
print("\n(Para detener el servidor: interrupt/stop kernel o Ctrl+C en la consola donde corre el notebook)\n")

app.run(host="127.0.0.1", port=5000, debug=False)

✅ index.html generado/actualizado en: C:\Users\Jorge Vasquez\DocoBr\index.html
📄 Plantilla esperada: C:\Users\Jorge Vasquez\DocoBr\main.docx
📁 Carpeta de salida: C:\Users\Jorge Vasquez\DocoBr\GENERADOS

🚀 Web lista. Abre en tu navegador:
   http://127.0.0.1:5000

(Para detener el servidor: interrupt/stop kernel o Ctrl+C en la consola donde corre el notebook)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
